In [2]:
import h5py
f = h5py.File('data.hdf5', 'r')

import numpy as np
print(list(f.keys()))
test = f['test']
test_data=np.array(test)
train = f['train']
train_data=np.array(train)
print(type(train))
print(np.shape(train))
print(train[0,0,0])


['test', 'train']
<class 'h5py._hl.dataset.Dataset'>
(30000, 1024, 2)
0.81229824


In [3]:
import pandas as pd
tl = pd.read_csv('train_labels.csv', delimiter = ',') #read dataset
tl.head(5)
tl_n = tl.to_numpy()
trn_y = tl_n[:,1]
trn_y[trn_y=='FM']=0
trn_y[trn_y=='OQPSK']=1
trn_y[trn_y=='BPSK']=2
trn_y[trn_y=='8PSK']=3
trn_y[trn_y=='AM-SSB-SC']=4
trn_y[trn_y=='4ASK']=5
trn_y[trn_y=='16PSK']=6
trn_y[trn_y=='AM-DSB-SC']=7
trn_y[trn_y=='QPSK']=8
trn_y[trn_y=='OOK']=9

trn_y=trn_y.astype('int')
print(np.shape(trn_y))
print(type(trn_y))
trn_y

(30000,)
<class 'numpy.ndarray'>


array([3, 4, 7, ..., 4, 5, 0])

In [ ]:
from scipy.stats import moment

def feature_extraction(given_data):
    
    train_data_cmplx = given_data[:,:,0] + 1j*given_data[:,:,1]
    data = np.fft.fft(train_data_cmplx,axis=1)
    
    dim = len(data)
    
    M20 = moment(data,2,axis=1)
    print(np.shape(M20))

    M21 = data * np.conj(data)
    M21 = moment(M21, 1, axis=1)
    print(np.shape(M21))

    M22 = np.power(data,0) * np.power(np.conj(data),2)
    M22 = moment(M22, 1, axis=1)
    print(np.shape(M22))

    M40 = np.power(data,4) * np.power(np.conj(data),0)
    M40 = moment(M40, 1, axis=1)
    print(np.shape(M40))

    M41 = np.power(data,4) * np.power(np.conj(data),0)
    M41 = moment(M41, 1, axis=1)
    print(np.shape(M41))

    M42 = np.power(data,4) * np.power(np.conj(data),0)
    M42 = moment(M42, 1, axis=1)
    print(np.shape(M42))

    M43 = np.power(data,4) * np.power(np.conj(data),0)
    M43 = moment(M43, 1, axis=1)
    print(np.shape(M43))

    M60 = np.power(data,6) * np.power(np.conj(data),0)
    M60 = moment(M60, 1, axis=1)
    print(np.shape(M60))

    M61 = np.power(data,5) * np.power(np.conj(data),1)
    M61 = moment(M61, 1, axis=1)
    print(np.shape(M61))

    M62 = np.power(data,4) * np.power(np.conj(data),2)
    M62 = moment(M62, 1, axis=1)
    print(np.shape(M62))

    M63 = np.power(data,3) * np.power(np.conj(data),3)
    M63 = moment(M63, 1, axis=1)
    print(np.shape(M63))


    C40 = M40 - 3*np.power(M20,2)
    print(np.shape(C40))

    C41 = M40 - 3*M20 * M21
    print(np.shape(C41))

    C42 = M42 - np.power(np.absolute(M20),2) - 2*np.power(M21,2)
    print(np.shape(C42))

    C60 = M60 - 16*M20*M40 + 30*np.power(np.absolute(M20),2)
    print(np.shape(C60))

    C61 = M61 - 5*M21*M40 - 10*M20*M41 + 30*np.power(np.absolute(M20),2)*M21
    print(np.shape(C61))

    C62 = M62 - 6*M20*M42 - 8*M21*M41 -M22*M40 + 6*np.power(np.absolute(M20),2)*M22 + 24*np.power(np.absolute(M21),2)*M20 
    print(np.shape(C62))

    C63 = M63 - 9*M21*M42 + 12*np.power(M21,3) - 3*M20*M43 - 3*M22*M41 + 18*M20*M21*M22
    print(np.shape(C63))
    
    output = [M20, M21, M22, M40, M41, M42, M43, M60, M61, M62, M63, C40, C41, C42, C60, C61, C62, C63]
    output = np.reshape(output,(dim,18))
    
    output_real = np.real(output)
    output_imag = np.imag(output)
    
    output = [output_real,output_imag]
    output = np.reshape(output,(dim,36))
    
    return output
    

In [ ]:
train_features=feature_extraction(train_data)
print(np.shape(train_features))
print(type(train_features))



In [ ]:
from sklearn.svm import SVC

clf = SVC(C=10,kernel='rbf',gamma='auto')
clf.fit(train_features,trn_y)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


train_x = np.reshape(train_data,(30000,2048))
clf = SVC(C=10,kernel='poly',degree=2,gamma='auto')
clf.fit(train_x[0:10000,:],trn_y[0:10000])
y_new_trn=clf.predict(train_x[0:10000,:])
accuracy_trn=accuracy_score(y_new_trn, trn_y[0:10000])
print(accuracy_trn)

In [ ]:
accuracy_tst = []
accuracy_trn = []
n = np.array(range(20))
C = 2**n
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

for i in range(20):
    clf = SVC(C=C[i],kernel='poly',degree=4,gamma='auto')
    clf.fit(train_features[0:30000,:],trn_y[0:30000])
    y_new_trn=clf.predict(train_features[0:30000,:])
    accuracy_trn=np.append(accuracy_trn,accuracy_score(y_new_trn, trn_y[0:30000]))

max_acc = np.amax(accuracy_trn)
#plt.semilogx(C, accuracy_tst)
plt.semilogx(C, accuracy_trn)
plt.title("SVM with Gaussian kernel")
plt.xlabel('C')
plt.ylabel('accuracy')
plt.show()
print('The maximum testing accuracy achieved with SVM with Gaussian kernel is: ' + str(max_acc))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(random_state=0,max_depth=100,learning_rate=0.2)
clf.fit(train_features[0:10000], trn_y[0:10000])
from sklearn.metrics import accuracy_score
y_new_trn=clf.predict(train_features[0:10000,:])
accuracy_trn=accuracy_score(y_new_trn,trn_y[0:10000])
print(accuracy_trn)

In [ ]:
def normalise(given_data):
    dim=len(given_data)
    data_comp=given_data[:,:,0] + 1j*given_data[:,:,1]
    data = np.zeros((dim,1024,2))
    data[:,:,0] = np.absolute(data_comp)
    data[:,:,1] = np.angle(data_comp)
    print(np.shape(data))
    print(data[0])
    data=given_data
    for i in range(dim):
        for j in range(2):
            mu = np.mean(data[i,:,j])
            data[i,:,j] = data[i,:,j]-np.mean(data[i,:,j])
            std=np.var(data[i,:,j])
            data[i,:,:] = data[i,:,:]/std
    return data
trn_norm = normalise(x_trn)
print(np.shape(trn_norm))

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras import backend as K
from tensorflow.keras import regularizers


batch_size = 300
num_classes = 24
epochs = 500
input_shape = (1024,2)
fil = 64
ker=2


y_train = keras.utils.to_categorical(trn_y, num_classes)

#Input,output and hidden layers
model = Sequential()
model.add(Conv1D(fil, kernel_size= ker,
                 activation='relu', padding="same",
                 input_shape=input_shape))

#Reduces the input data to half
model.add(MaxPooling1D(pool_size=(2)))
model.add(Conv1D(fil, ker,padding="same", activation='relu'))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Conv1D(fil, ker,padding="same", activation='relu'))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Conv1D(fil, ker,padding="same", activation='relu'))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Conv1D(fil, ker,padding="same", activation='relu'))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Conv1D(fil, ker,padding="same",activation='relu'))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Conv1D(fil, ker,padding="same", ,activation='relu'))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Dropout(0.25))
model.add(Flatten())
#model.add(Flatten())
model.add(Dense(128, activation='selu'))
model.add(Dense(128, activation='selu'))
#model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


model.compile(loss = keras.losses.categorical_crossentropy,
              optimizer = keras.optimizers.Adam(),
              metrics = ['accuracy'])

#Model train in batches
model.summary()
model.fit(trn_norm, y_train, batch_size = batch_size, epochs = epochs)

In [ ]:
tst_norm = normalise(x_test)
y_pred = model.predict(tst_norm)



print(np.shape(y_pred))

label = np.zeros(20000)
for i in range (0,19999):
    m = np.argmax(y_pred[i][:])
    label[i] = m.astype(int)
test_y=label

In [ ]:
test_features=feature_extraction(test_data)
tst_y=clf.predict(test_features)

print(np.shape(tst_y))

test_y=np.empty(np.shape(tst_y), dtype=object)
for i in range(19999):
  #print(i)
    if(tst_y[i]==0):
            test_y[i]='FM'
    if(tst_y[i]==1):
        test_y[i]='QPSK'
    if(tst_y[i]==2):
        test_y[i]='BPSK'
    if(tst_y[i]==3):
        test_y[i]='8PSK'
    if(tst_y[i]==4):
        test_y[i]='AM-SSB-SC'
    if(tst_y[i]==5):
        test_y[i]='4ASK'
    if(tst_y[i]==6):
        test_y[i]='16PSK'
    if(tst_y[i]==7):
        test_y[i]='AM-DSB-SC'
    if(tst_y[i]==8):
        test_y[i]='QPSK'
    if(tst_y[i]==9):
        test_y[i]='OOK'

print(np.shape(test_y))
ID = np.linspace(0,19999,num=20000)
test_label = test_y
print(np.shape(test_label))
test_label=np.reshape(test_label,(20000))
test_csv = pd.DataFrame(data=test_label,columns=["Category"])
test_csv.to_csv('test_svm.csv')